## Prepare the runtime environment

In [ ]:
# Clone Repo
!git clone https://github.com/shariqahn/EasyEdit.git
%cd EasyEdit
!ls

Cloning into 'EasyEdit'...
remote: Enumerating objects: 7751, done.
remote: Counting objects: 100% (1768/1768), done.
remote: Compressing objects: 100% (406/406), done.
remote: Total 7751 (delta 1539), reused 1411 (delta 1362), pack-reused 5983 (from 3)
Receiving objects: 100% (7751/7751), 72.52 MiB | 35.77 MiB/s, done.
Resolving deltas: 100% (5083/5083), done.
/EasyEdit
Dockerfile		 tofu.sh.log-27575665
LICENSE			 tofu.sh.log-27575825
README.md		 tofu.sh.log-27579879
colab_requirements.txt	 tofu.sh.log-27681647
data			 tofu.sh.log-dummy_ground_truth
debug.py		 tofu.sh.log-lr_1_dummy_rome
demo			 tofu.sh.log-rome-clamp-1
download.py		 tofu.sh.log-rome-dummy-seq
download_models.sh	 tofu.sh.log-rome-incorrect
easyeditor		 tofu.sh.log-rome-incorrect-clamp-2
edit.py			 tofu.sh.log-rome-incorrect-lr-1
examples		 tofu.sh.log-serac-dummy
figs			 train.sh
formatted_sentences.txt  train.sh.log-27634535
hparams			 train.sh.log-27634568
install_log.txt		 train.sh.log-27662286
multimodal_edit.py	 

/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import sys
print("Python executable:", sys.executable)
print("Python version:", sys.version)


Python executable: /usr/bin/python
Python version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [ ]:
# Verify the installation
!python3 --version
!python --version
!pip --version

Python 3.10.12
Python 3.10.12
pip 23.3.1 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [4]:
# takes ~2 mins
!pip install --no-cache-dir -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 298.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 273.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 235.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 202.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 3.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 303.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip list


In [ ]:
!pip install matplotlib
!pip install sentence_transformers

## Import modules & Run

In [ ]:
%mkdir /workspace/hf
!ls /workspace
%pwd

In [ ]:
experiment="baseline"
# data="./data/tofu_test_dummy_zsre.json"
# data="./data/tofu_test_zsre.json"
# data="./data/tofu_test_avoidant_zsre.json"
data="./data/notebook/zsre_mend_eval_portability_gpt4.json"
metrics_save_dir=f"/workspace/outputs/MEMIT_{experiment}/"

### For LlaMA Model

In [ ]:
from easyeditor import BaseEditor
from easyeditor import MEMITHyperParams
import json

In [ ]:
test_data = json.load(open(data, 'r', encoding='utf-8'))
prompts = [test_data_['src'] for test_data_ in test_data]
ground_truth = [edit_data_['pred'] for edit_data_ in test_data]
rephrase_prompts = [edit_data_['rephrase'] for edit_data_ in test_data]
target_new = [edit_data_['alt'] for edit_data_ in test_data]
locality_prompts = [edit_data_['loc'] for edit_data_ in test_data]
locality_ans = [edit_data_['loc_ans'] for edit_data_ in test_data]

locality_inputs = {
    'neighborhood':{
        'prompt': locality_prompts,
        'ground_truth': locality_ans
    },
}

if experiment == "baseline":
    portability_prompts = [edit_data_['portability']['New Question'] for edit_data_ in test_data]
    portability_ans = [edit_data_['portability']['New Answer'] for edit_data_ in test_data]
    portability_inputs = {
        'one_hop':{
            'prompt': portability_prompts,
            'ground_truth': portability_ans
        },
    }

subject = [edit_data_['subject'] for edit_data_ in test_data]\

In [ ]:
# ~13m
import os

hparams=MEMITHyperParams.from_hparams('./hparams/MEMIT/notebook.yaml')
editor = BaseEditor.from_hparams(hparams)

sequential_edit = True
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    rephrase_prompts=rephrase_prompts,
    target_new=target_new,
    subject=subject,
    locality_inputs=locality_inputs,
    portability_inputs=portability_inputs,
    keep_original_weight=False,
    sequential_edit=sequential_edit
)

print('data: ', data)
print('save to: ', metrics_save_dir)
print('model: ', hparams.model_name)
print('sequential_edit: ', sequential_edit)

os.makedirs(metrics_save_dir, exist_ok=True)
json.dump(metrics, open(os.path.join(metrics_save_dir, f'MEMIT_results.json'), 'w'), indent=4)
model_save_dir = os.path.join(metrics_save_dir, 'model')
os.makedirs(model_save_dir, exist_ok=True)
edited_model.save_pretrained(model_save_dir)

In [ ]:
# Save model on SuperCloud
!ssh-keyscan txe1-login.mit.edu >> ~/.ssh/known_hosts
!ssh-keygen -t rsa -b 2048 -f ~/.ssh/id_rsa -N ""
!cat ~/.ssh/id_rsa.pub

In [ ]:
# ~8m
!apt update -y
!apt install rsync -y
!rsync -avz -e ssh /workspace/outputs/ shossain@txe1-login.mit.edu:/home/gridsan/shossain/EasyEdit/outputs/ -y

In [ ]:
# TODO do something with stats_dir in hparams?